In [10]:
# TODO 出力カラムのズレとD3補正しない値()、相互作用エネの計算(ドライブにある)

import os
import csv

# フォルダのパスを指定
folder_path = 'data/SPA/Type_2_ck_at_E/B2_B_CP/'

# CSVファイルのパス
output_csv_path = folder_path.replace("data", "result")
os.makedirs(output_csv_path, exist_ok=True)

# CSVファイルのヘッダー
headers = ['File Name', 'count', 'SCF Done Value (D3-corrected)', 'Grimme-D3 Dispersion Energy', 'SCF Done Value (B3LYP)']

# CSVファイルに書き込むためのデータを格納するリスト
csv_data = []

# フォルダ内の全ファイルを走査
for filename in os.listdir(folder_path):
    # .out ファイルのみを処理
    if filename.endswith('.out'):
        file_path = os.path.join(folder_path, filename)

        # ファイルを開いて必要なデータを抽出
        with open(file_path, 'r') as file:
            scf_done_values = []
            grimme_d3_values = []

            for line in file:
                if 'SCF Done:  E(RB3LYP)' in line:
                    # SCF Doneの値を抽出
                    scf_done = float(line.split('=')[1].split('A.U.')[0].strip())
                    scf_done_values.append(scf_done)
                elif 'Grimme-D3 Dispersion energy=' in line:
                    # Grimme-D3 Dispersion energyの値を抽出
                    grimme_d3 = float(line.split('=')[1].split("Hartrees")[0].strip())
                    grimme_d3_values.append(grimme_d3)

            # SCF DoneとGrimme-D3 Dispersion energyの値を組み合わせてCSVデータを作成
            for i, (scf_done, grimme_d3) in enumerate(zip(scf_done_values, grimme_d3_values)):
                adjusted_scf_done = scf_done - grimme_d3
                csv_data.append([filename, i+1, scf_done, grimme_d3, adjusted_scf_done])     
                
            
output_csv_path = output_csv_path + filename.replace("out", "csv")
# CSVファイルにデータを書き込む
with open(output_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    writer.writerows(csv_data)
